<a href="https://colab.research.google.com/github/jiiwon129/ESAA/blob/main/ESAA_YB_WEEK11_Dacon_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **대구 교통사고 피해 예측 AI 경진대회**

https://dacon.io/competitions/official/236193/overview/description


참고 수상작:

https://dacon.io/competitions/official/236193/codeshare/9496?page=1&dtype=recent

#### **주제**

시공간 정보로부터 사고위험도(ECLO) 예측 AI 모델 개발

- ECLO(Equivalent Casuality Loss Only): 인명피해 심각도
  - ECLO = 사망자 수 * 10 + 중상자 수 * 5 + 경상자 수 * 3 + 부상자 수 * 1
  - 본 대회에서 사고의 위험도를 인명피해 심각도로 측정

#### **데이터 설명**

- 시간적 정보: 요일 및 날짜
- 공간적 정보: 해당 위치, 노면상태, 기상상황, 도로유형

- target = ECLO(인명피해 심각도)

#### **Validation Strategy**

- 공공데이터 포털에서 대회에서 제공되는 train 데이터뿐만 아니라 대구 사고 데이터를 추가적으로 수집하고 전처리하여 훈련 데이터로 활용
  - 모델의 예측 안정성 및 일반화 성능 향상

- 대회에서 제공하는 대구 내의 보안등, 어린이 보호구역, 주차장, CCTV 외부데이터를 활용하여 여러 피처 생성
  - 모델의 예측 정확도 향상

- 사망자 수 중상자 수, 경상자 수, 부상자 수를 활용하여 대구광역시 구 및 동별로 사고 위험도를 학습하게 피처 생성

- 날짜 정보를 이용해 년도, 월, 일, 요일, 시간 파생변수 및 Cycling 파생변수 등 시간 정보를 담은 파생변수 생성 후
XGBoost로 학습
  - 시계열 정보를 학습할 수 있는 XGBoost를 사용함으로써 예측 정확도 향상

- Validation set을 Stratified K-Fold 방식 사용
  - target 값이 골고루 분포하게 구성
  - 모델의 예측 정확도 향상
  - 추가로 K(나누는 Fold의 수) 값을 다양하게 구성
    - 예측의 일반화 성능 향상

- 모델별로 Seed를 다양하게 구성
  - 예측의 일반화 성능 향상

#### **EDA**

시공간적으로 어떤 부분이 target값 ECLO에 영향을 미치는 지에 집중
1. 새벽 이외의 시간대(6시-)에는 낮다가 새벽 시간대(0-5시)에 높아짐
2. 평일에는 대체적으로 낮고, 토요일부터 높아져서 일요일에 가장 높은 값을 보임
3. 특정 구(달성군, 동구)에서 다른 구에 비해 높은 사고 위험도를 보임
  - 구별로 ECLO값의 평균을 비교하여 도출
4. 사고 지역 근처에 대부분 고속도로가 있거나, 산간지역에 위치해 있다는 특성을 지님
  - 동별로 ECLO값의 평균을 비교하여 도출
5. 대체적으로 제한속도가 높아질수록 사고 위험도가 높아짐을 확인
  - 설치된 CCTV 최대 제한 속도 및 최소 제한 속도별로 ECLO값 비교

#### **Feature Engineering**

1. 0시부터 23시까지로 구성된 시간 feature를 시간대별로 grouping
2. 평일, 토요일, 일요일을 오름차순으로 Label encoding
3. 시간 및 날짜 feature들의 inherently cyclical 하다는 특징을 활용하기 위해 sin/cos 변환
4. 구별로 ECLO값이 평균이 5보다 큰 구를 뽑아 그 구에 속하는 지 여부에 대한 파생변수 생성
5. 동별로 ECLO값의 평균이 5보다 큰 동을 뽑아 그 동에 속하는 지 여부에 대한 파생변수 생성
6. 동별로 사망자, 중상자, 경상자 및 부상자 수를 sum하는 파생변수 생성

#### **Modeling**

1. CatBoost 사용
  - 범주형 변수로 구성된 공간정보를 활용
2. XGBoost 사용
  - 날짜 및 시간 정보를 시계열 정보로 활용
3. Overfitting을 방지하기 위해 별도의 파라미터 튜닝은 하지 않음
4. 여러 개의 교차 검증 활용(Multi-Stratified-K-Fold cross validation)
5. 다양한 Seed를 활용

#### **차별점 및 배울점**

- target값을 결정 지을 수 있는 핵심 정보가 부족해 외부데이터를 매우 다양하게 수집 및 활용한 점이 눈에 띄었음
  - 공공데이터 포털에서 주어진 train 데이터와 비슷한 형태의 데이터를 따로 수집하여 활용한 점

- 주어진 데이터 내에서 다양한 새로운 파생변수 및 피처 생성

- EDA를 매우 디테일하게 진행하고 분석함

- 시공간적 정보를 다루기 위해 공간적 정보는 범주형으로 CatBoost, 시간적 정보는 시계열 정보 활용에 유리한 XGBoost을 사용한 점

- 추가적으로 Overfitting을 방지하기 위해 별도의 파라미터 튜닝을 진행하지 않은 점
  - 예측의 정확도를 높이기 위해서는 하이퍼 파라미터 튜닝이 거의 필수적이라 생각했었는데, 이 생각을 바꾸게 된 계기가 됨

